In [1]:
# Clean uninstall just in case
!pip uninstall -y torch-scatter torch-sparse torch-geometric pyg-lib

# Install compatible packages for torch 2.5.1 + CUDA 12.1
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install -q pyg-lib -f https://data.pyg.org/whl/torch-2.5.1+cu121.html
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 87.0 MB/s eta 0:00:00:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 40.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.6 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00a 0:00:01


In [2]:
import argparse
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
!pip install torch-geometric
from torch_geometric.nn import GCNConv, SAGEConv, GATConv, APPNP
from torch_geometric.loader import NeighborLoader

TODO:
Implement teacher network architecture and training along with flags for datasets and teacher model architecture (Chanikya and Nithin)
eg: python3 train_teacher.py --dataset=cora --model=SAGE --epochs-100 --lr=0.01 . Add flags for other hyperparameters if necessary (Chanikya and Nithin)
Other teacher model architectures - GCN, GAT, APPNP (Chanikya and Nithin + others based on availability)

In [3]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='./Cora', name='Cora')

data = dataset[0]
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Processing...


Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7


Done!


In [4]:
# data.edge_index.t()

In [5]:
def load_data(dataset):
    if dataset == "cora":
        dataset = Planetoid(root='./Cora', name='Cora')
        data = dataset[0]
        graph_nx = nx.Graph()
        graph_nx.add_edges_from(data.edge_index.t().tolist())

        # Adding self-loops
        # graph_nx.add_edges_from((n, n) for n in graph_nx.nodes())
        
        # adj_tensor = torch.tensor(nx.to_numpy_array(graph_nx), dtype=torch.float).to('cuda')
        features = data.x
        labels = data.y

        train_idx = data.train_mask.nonzero(as_tuple=True)[0]
        val_idx = data.val_mask.nonzero(as_tuple=True)[0]
        test_idx = data.test_mask.nonzero(as_tuple=True)[0]
        
        return data.edge_index, features, labels, data.train_mask, data.val_mask, data.test_mask

In [18]:
edge_index, features, labels, train_idx, val_idx, test_idx = load_data("cora")


GCN => num layers, hidden, input dim, out, dp, activa,  

In [6]:
class GCN(nn.Module):
    def __init__(
        self,
        num_layers,
        input_dim,
        hidden_dim,
        output_dim,
        dropout_ratio,
        activation,
        norm_type="none"
    ):
        super().__init__()
        self.num_layers = num_layers
        self.norm_type = norm_type
        self.dropout = nn.Dropout(dropout_ratio)
        self.activation = activation

        self.layers = nn.ModuleList()
        self.norms = nn.ModuleList()

        if num_layers == 1:
            self.layers.append(GCNConv(input_dim, output_dim))
        else:
            self.layers.append(GCNConv(input_dim, hidden_dim))
            if norm_type == "batch":
                self.norms.append(nn.BatchNorm1d(hidden_dim))
            elif norm_type == "layer":
                self.norms.append(nn.LayerNorm(hidden_dim))

            for _ in range(num_layers - 2):
                self.layers.append(GCNConv(hidden_dim, hidden_dim))
                if norm_type == "batch":
                    self.norms.append(nn.BatchNorm1d(hidden_dim))
                elif norm_type == "layer":
                    self.norms.append(nn.LayerNorm(hidden_dim))

            self.layers.append(GCNConv(hidden_dim, output_dim))

    def forward(self, x, edge_index):
        h_list = []
        h = x
        for l, layer in enumerate(self.layers):
            h = layer(h, edge_index)
            if l != self.num_layers - 1:
                if self.norm_type != "none":
                    h = self.norms[l](h)
                h = self.activation(h)
                h = self.dropout(h)
                h_list.append(h)
        return h_list[-1], h

In [7]:
class SAGE(nn.Module):
    def __init__(
        self,
        num_layers,
        input_dim,
        hidden_dim,
        output_dim,
        dropout_ratio,
        activation,
        norm_type="none",
    ):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.norm_type = norm_type
        self.activation = activation
        self.dropout = nn.Dropout(dropout_ratio)
        self.layers = nn.ModuleList()
        self.norms = nn.ModuleList()

        if num_layers == 1:
            self.layers.append(SAGEConv(input_dim, output_dim))
        else:
            self.layers.append(SAGEConv(input_dim, hidden_dim))
            if self.norm_type == "batch":
                self.norms.append(nn.BatchNorm1d(hidden_dim))
            elif self.norm_type == "layer":
                self.norms.append(nn.LayerNorm(hidden_dim))

            for _ in range(num_layers - 2):
                self.layers.append(SAGEConv(hidden_dim, hidden_dim))
                if self.norm_type == "batch":
                    self.norms.append(nn.BatchNorm1d(hidden_dim))
                elif self.norm_type == "layer":
                    self.norms.append(nn.LayerNorm(hidden_dim))

            self.layers.append(SAGEConv(hidden_dim, output_dim))

    def forward(self, x, edge_index):
        h = x
        h_list = []
        for l, layer in enumerate(self.layers):
            h = layer(h, edge_index)
            if l != self.num_layers - 1:
                h_list.append(h)
                if self.norm_type != "none":
                    h = self.norms[l](h)
                h = self.activation(h)
                h = self.dropout(h)
        return h_list[-1], h


    def inference(self, x_all, edge_index, batch_size=1024, device="cuda"):
        """
        Full-graph inference using mini-batches (for large graphs).
        """
        from torch_geometric.loader import NeighborLoader

        x = x_all.to(device)
        for l, layer in enumerate(self.layers):
            new_x = torch.zeros(
                x_all.size(0),
                self.hidden_dim if l != self.num_layers - 1 else self.output_dim,
            ).to(device)

            loader = NeighborLoader(
                data=(x_all, edge_index),
                input_nodes=torch.arange(x_all.size(0)),
                num_neighbors=[-1],  # full neighbors
                batch_size=batch_size,
                shuffle=False
            )

            for batch in loader:
                batch = batch.to(device)
                h = x[batch.n_id]
                h = layer(h, batch.edge_index)

                if l != self.num_layers - 1:
                    if self.norm_type != "none":
                        h = self.norms[l](h)
                    h = self.activation(h)
                    h = self.dropout(h)

                new_x[batch.n_id[:batch.batch_size]] = h

            x = new_x
        return x
# For small, medium datasets few thousands, use model() in eval
# For large ones like 100k or millions, use inference

In [8]:
class GAT(nn.Module):
    def __init__(
            self,
            num_layers,
            input_dim,
            hidden_dim,
            output_dim,
            dropout_ratio,
            activation=F.relu,
            num_heads=8,
            attn_drop=0.3,
            negative_slope=0.2,
            residual=False,
    ):
        super().__init__()
        
        assert num_layers > 1

        hidden_dim //= num_heads 
        self.num_layers = num_layers
        self.layers = nn.ModuleList()
        self.activation = activation
        self.dropout = nn.Dropout(dropout_ratio)

        heads = [num_heads] * (num_layers - 1) + [1]  
        # heads = ([num_heads] * num_layers) + [1]

        # Input layer
        self.layers.append(
            GATConv(
                in_channels=input_dim,
                out_channels=hidden_dim,
                heads=heads[0],
                dropout=attn_drop,
                negative_slope=negative_slope,
                concat=True, 
            )
        )

        # Hidden layers
        for l in range(1, num_layers - 1):
            self.layers.append(
                GATConv(
                    in_channels=hidden_dim * heads[l - 1],  
                    out_channels=hidden_dim,
                    heads=heads[l],
                    dropout=attn_drop,
                    negative_slope=negative_slope,
                    concat=True, 
                )
            )

        # Output layer
        self.layers.append(
            GATConv(
                in_channels=hidden_dim * heads[-2],  
                out_channels=output_dim,
                heads=heads[-1],  
                dropout=attn_drop,
                negative_slope=negative_slope,
                concat=False, 
            )
        )

    def forward(self, x, edge_index):
        h_list = []
        h = x
        for l, layer in enumerate(self.layers):
            h = self.dropout(h) 
            h = layer(h, edge_index)
            if l != self.num_layers - 1:
                h = self.activation(h)  
                h_list.append(h)
        return h_list[-1], h

In [9]:
class APPNP_Model(nn.Module):
    def __init__(
            self,
            num_layers,
            input_dim,
            hidden_dim,
            output_dim,
            dropout_ratio,
            activation=F.relu,
            norm_type="none",
            edge_drop=0,
            alpha=0.1,
            k=10,
    ):
        super().__init__()
        self.num_layers = num_layers
        self.norm_type = norm_type
        self.activation = activation
        self.dropout = nn.Dropout(dropout_ratio)
        self.layers = nn.ModuleList()
        self.norms = nn.ModuleList()

        # Input layer
        if num_layers == 1:
            self.layers.append(nn.Linear(input_dim, output_dim))
        else:
            self.layers.append(nn.Linear(input_dim, hidden_dim))
            if self.norm_type == "batch":
                self.norms.append(nn.BatchNorm1d(hidden_dim))
            elif self.norm_type == "layer":
                self.norms.append(nn.LayerNorm(hidden_dim))

            # Hidden layers
            for _ in range(num_layers - 2):
                self.layers.append(nn.Linear(hidden_dim, hidden_dim))
                if self.norm_type == "batch":
                    self.norms.append(nn.BatchNorm1d(hidden_dim))
                elif self.norm_type == "layer":
                    self.norms.append(nn.LayerNorm(hidden_dim))

            # Output layer
            self.layers.append(nn.Linear(hidden_dim, output_dim))

        self.propagate = APPNP(K=k, alpha=alpha, dropout=edge_drop)
        self.reset_parameters()

    def reset_parameters(self):
        for layer in self.layers:
            if hasattr(layer, "reset_parameters"):
                layer.reset_parameters()

    def forward(self, x, edge_index):
        h_list = []
        h = x

        for l, layer in enumerate(self.layers):
            h = layer(h)

            if l != self.num_layers - 1:  
                h_list.append(h)
                if self.norm_type != "none":
                    h = self.norms[l](h)
                h = self.activation(h)
                h = self.dropout(h)

        h = self.propagate(h, edge_index)
        return h_list[-1], h 

In [10]:
def train_sage(model, loader, optimizer, criterion, device, homo=True):
    model.train()
    total_loss = 0

    for batch in loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        x = batch.x
        y = batch.y[:batch.batch_size]  # Only use input nodes

        if homo:
            edge_index = batch.edge_index
        else:
            rel = list(batch.edge_index_dict.keys())[0]
            edge_index = batch.edge_index_dict[rel]

        _, out = model(x, edge_index)
        out = out[:batch.batch_size]  # Only use predictions for input nodes

        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)


@torch.no_grad()
def evaluate_sage(model, loader, device, homo=True):
    model.eval()
    correct = 0
    total = 0

    for batch in loader:
        batch = batch.to(device)
        x = batch.x
        y = batch.y[:batch.batch_size]  # Only input nodes

        if homo:
            edge_index = batch.edge_index
        else:
            rel = list(batch.edge_index_dict.keys())[0]
            edge_index = batch.edge_index_dict[rel]

        _, out = model(x, edge_index)
        out = out[:batch.batch_size]  # Only predictions for input nodes

        pred = out.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    return correct / total

def train(model, data, edge_index, labels, train_idx, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    _, out = model(data, edge_index)
    loss = criterion(out[train_idx], labels[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(model, data, edge_index, labels, idx):
    model.eval()
    _, out = model(data, edge_index)
    pred = out[idx].argmax(dim=1)
    correct = (pred == labels[idx]).sum().item()
    acc = correct / sum(idx)
    return acc


In [21]:
    
def run_SAGE():    
    train_loader = NeighborLoader(
        data,
        input_nodes=data.train_mask,
        num_neighbors=[15, 10],
        batch_size=128,
        shuffle=True
    )
    
    val_loader = NeighborLoader(
        data,
        input_nodes=data.val_mask,
        num_neighbors=[-1, -1],
        batch_size=128
    )
    
    test_loader = NeighborLoader(
        data,
        input_nodes=data.test_mask,
        num_neighbors=[-1, -1],
        batch_size=128
    )
    
    
    model = SAGE(
        num_layers=2,
        input_dim=dataset.num_node_features,
        hidden_dim=128,
        output_dim=dataset.num_classes,
        dropout_ratio=0,
        activation=nn.functional.relu,
        norm_type="batch"
    )
    
    device = 'cuda'
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()
    
    for epoch in range(1, 201):
        loss = train_sage(model, train_loader, optimizer, criterion, device)
        val_acc = evaluate_sage(model, val_loader, device)
        if epoch % 10 == 0 or epoch == 1:
            test_acc =evaluate_sage(model, test_loader, device)
            print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")



In [13]:
def run_GCN():
    model = GCN(
        num_layers=3,
        input_dim=dataset.num_node_features,
        hidden_dim=64,
        output_dim=dataset.num_classes,
        dropout_ratio=0.8,
        activation=nn.functional.relu,
        norm_type="batch"
    )
    # model = GCN1(dataset.num_node_features, 64, dataset.num_classes)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()
    
    for epoch in range(1, 150):
        loss = train(model, features, edge_index, labels, train_idx, optimizer, criterion)
        val_acc = evaluate(model, features, edge_index, labels, val_idx)
        if epoch % 10 == 0 or epoch == 1:
            test_acc = evaluate(model, features, edge_index, labels, test_idx)
            print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")


In [14]:
def run_APPNP():
    model = APPNP_Model(
        num_layers=2,
        input_dim=dataset.num_node_features,  
        hidden_dim=128,
        output_dim=dataset.num_classes,  
        dropout_ratio=0.5,
        activation=F.relu,
    )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    
    for epoch in range(1, 100):
        loss = train(model, features, edge_index, labels, train_idx, optimizer, criterion)
        val_acc = evaluate(model, features, edge_index, labels, val_idx)
        if epoch % 10 == 0 or epoch == 1:
            test_acc = evaluate(model, features, edge_index, labels, test_idx)
            print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")

In [15]:
def run_GAT():
    model = GAT(
        num_layers=2,
        input_dim=dataset.num_node_features,  
        hidden_dim=128,
        output_dim=dataset.num_classes,       
        dropout_ratio=0.6,
        activation=F.relu,
        num_heads=8,
        attn_drop=0.3,
        negative_slope=0.2,
        residual=True
    )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    
    for epoch in range(1, 150):
        loss = train(model, features, edge_index, labels, train_idx, optimizer, criterion)
        val_acc = evaluate(model, features, edge_index, labels, val_idx)
        if epoch % 10 == 0 or epoch == 1:
            test_acc = evaluate(model, features, edge_index, labels, test_idx)
            print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f} | Test Acc: {test_acc:.4f}")

In [22]:
run_SAGE()

Epoch 001 | Loss: 1.8793 | Val Acc: 0.2860 | Test Acc: 0.2930
Epoch 010 | Loss: 0.0134 | Val Acc: 0.7280 | Test Acc: 0.7510
Epoch 020 | Loss: 0.0026 | Val Acc: 0.7580 | Test Acc: 0.7640
Epoch 030 | Loss: 0.0020 | Val Acc: 0.7520 | Test Acc: 0.7630
Epoch 040 | Loss: 0.0016 | Val Acc: 0.7620 | Test Acc: 0.7670
Epoch 050 | Loss: 0.0016 | Val Acc: 0.7580 | Test Acc: 0.7710
Epoch 060 | Loss: 0.0009 | Val Acc: 0.7620 | Test Acc: 0.7760
Epoch 070 | Loss: 0.0008 | Val Acc: 0.7560 | Test Acc: 0.7680
Epoch 080 | Loss: 0.0007 | Val Acc: 0.7600 | Test Acc: 0.7710
Epoch 090 | Loss: 0.0008 | Val Acc: 0.7580 | Test Acc: 0.7750
Epoch 100 | Loss: 0.0008 | Val Acc: 0.7640 | Test Acc: 0.7720
Epoch 110 | Loss: 0.0006 | Val Acc: 0.7580 | Test Acc: 0.7680
Epoch 120 | Loss: 0.0007 | Val Acc: 0.7580 | Test Acc: 0.7730
Epoch 130 | Loss: 0.0006 | Val Acc: 0.7520 | Test Acc: 0.7760
Epoch 140 | Loss: 0.0006 | Val Acc: 0.7580 | Test Acc: 0.7740
Epoch 150 | Loss: 0.0005 | Val Acc: 0.7580 | Test Acc: 0.7740
Epoch 16

In [ ]:
class Teacher:
    def __init__(self, args):
        self.args = args
        pass
    def graph_split():
        pass
    def train_transductive():
        pass
    def train_inductive():
        pass
    

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Teacher implementation")
    parser.add_argument('--num_runs', type=int, default=1, help='Number of runs')
    parser.add_argument('--setting', type=str, choices=['trans', 'ind'], required=True, help='Setting type: trans or ind')
    parser.add_argument('--data_path', type=str, required=True, help='Path to the dataset')
    parser.add_argument('--model_name', type=str, default='SAGE', help='Name of the model(SAGE, GCN, GAT, APPNP)')
    parser.add_argument('--num_layers', type=int, default=2, help='Number of layers in the model')
    parser.add_argument('--hidden_dim', type=int, default=128, help='Hidden dimension size')
    parser.add_argument('--drop_out', type=float, default=0, help='Dropout rate')
    parser.add_argument('--batch_sz', type=int, default=512, help='Batch size')
    parser.add_argument('--learning_rate', type=float, default=0.01, help='Learning rate')
    parser.add_argument('--output_path', type=str, default='./output', help='Path to save output')
    
    args = parser.parse_args()

    